In [ ]:
from LibraryENGcopia import *
from tqdm.notebook import tqdm
import sklearn.preprocessing as ps
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from random import randint
from fastdtw import fastdtw
import copy
import pymc as pm
name_data = '2018-11-27T10-29-42POP 2 BL .h5'
name_data2 = '2018-11-27T12-03-02POP 2 stimulation.h5'
name_data3='2018-11-27T10-56-39MiP5 KA stimulation.h5'
complete_string='/Users/Gaia_1/Downloads/PoiProMEA-master/h5files/'+name_data
complete_string2='/Users/Gaia_1/Downloads/PoiProMEA-master/h5files/'+name_data2
complete_string3='/Users/Gaia_1/Downloads/PoiProMEA-master/h5files/'+name_data3

In [ ]:
#data = h5py.File(complete_string,'r')
#data = h5py.File(complete_string2,'r')
data = h5py.File(complete_string3,'r')

data_readings = data['Data']['Recording_0']['AnalogStream']['Stream_0']['ChannelData'][()]
info = data['Data']['Recording_0']['AnalogStream']['Stream_0']['InfoChannel'][()]
info_table = pd.DataFrame(info, columns = list(info.dtype.fields.keys()))
labels = info_table['Label']
readings = pd.DataFrame(data = data_readings.transpose(), columns = labels)
fs = 10000 #Sampling Frequency
print(readings.shape)

In [ ]:
readings.columns

In [ ]:
inizio=0
#len_data=128000 #192 secondi
len_data=len(readings)
prova_multi=readings.iloc[inizio:len_data, :59]
#prova=prova.drop([b'Ref'],axis=1)
prova=prova_multi[b'87']
ref=readings[b'Ref']
ref=ref[inizio:len_data]
#prova=readings.iloc[:150000, :5]
print(prova.shape,ref.shape)

In [ ]:
lowcut = 300
highcut = 3000
fs=10000
order=8
b,a=butter_bandpass(lowcut,highcut,fs,order=order)
filt_ref=filtfilt(b,a,ref)
filt_prova=filtfilt(b,a,prova)
f_prova = filt_prova - filt_ref
prova=f_prova
prova.shape
r_prova=prova.reshape(-1, 1)
scaler = StandardScaler()
s_prova = scaler.fit_transform(r_prova)
prova=s_prova
s_prova.shape

In [ ]:
pos, neg=find_all_spikes(prova)

firing rate baseline: 90
firing rate pop 2 stimulation: 87
firing rate pop 5 KA stimulation: 110

In [ ]:
len(pos)

In [ ]:
pos_cut,n_pos, neg_cut,n_neg = cut(pos,neg,prova)
savedp = copy.deepcopy(pos_cut)
savedn = copy.deepcopy(neg_cut)

In [ ]:
pos_cut=mask_cuts(pos_cut)
neg_cut= mask_cuts(neg_cut)

In [ ]:
x=randint(1,min(len(pos_cut),len(neg_cut)))

plt.plot(savedp[x])
plt.plot(pos_cut[x])
plt.show()
plt.plot(savedn[x])
plt.plot(neg_cut[x])
plt.show()

## DBSCAN

In [ ]:
epsilon=1e-100
min_s=5
leafsize=120
mcs=150
final_data_neg=dbscan_clustering(neg_cut,n_neg,len_data,mcs,min_s,leafsize)

In [ ]:
len(final_data_neg[0])

In [ ]:
np.shape(n_neg)

In [ ]:
final_data_neg[0][10633]

In [ ]:
k=0
extra=[]
for i,x in enumerate(n_neg):
    #if x==extra_clus[0][k]:
    if x==final_data_neg[0][k] and k<len(final_data_neg[0])-1:
        extra.append(neg_cut[i])
        k+=1

In [ ]:
np.shape(extra)

In [ ]:
extra_clus=dbscan_clustering(np.array(extra),final_data_neg[0],len_data,mcs,min_s,leafsize)

In [ ]:
final_data_pos=dbscan_clustering(pos_cut,n_pos,len_data,mcs,min_s,leafsize)

In [ ]:
unique_labels

In [ ]:
len(final_data[0])

In [ ]:
def dbscan_clustering(cut,spike_list,len_data,mcs,min_s,leafsize):
    from sklearn.cluster import KMeans
    from sklearn.cluster import DBSCAN, HDBSCAN
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    from sklearn.decomposition import FastICA
    from sklearn.metrics import silhouette_score
    from scipy.stats import kurtosis
    import skfuzzy as fuzz
    import numpy as np
    import math

    scale = StandardScaler()
    estratti_norm = scale.fit_transform(cut)
    print('Total spikes: ', estratti_norm.shape[0])
    n_comp=10
    pca = PCA(n_components=n_comp)
    transformed = pca.fit_transform(estratti_norm)
    print('transformed')
    #transformed=cut

    #dbscan = DBSCAN(eps=epsilon, min_samples=min_s, leaf_size=leafsize)
    dbscan = HDBSCAN(min_cluster_size=mcs, min_samples=min_s, leaf_size=leafsize)
    labels = dbscan.fit_predict(transformed)


    final_data=[]
    unique_labels = np.unique(labels)

    if len(unique_labels) == 1:
        print("DBSCAN assigned only one cluster.")
    else:
        silhouette_avg = silhouette_score(transformed, labels)
        num_clusters = len(np.unique(labels[labels != -1]))
        print("For", num_clusters,"clusters, the silhouette score is:", silhouette_avg)

    fig = plt.figure(figsize=(8, 10))

    # Iterate over unique cluster labels
    for i, cluster_label in enumerate(unique_labels):
        # Extract data points for the current cluster
        cluster_data = cut[labels == cluster_label]
        #final_data.append(spike_list[labels == cluster_label].tolist())

        # Plot the individual cluster data
        quad = math.ceil(len(unique_labels))
        plt.subplot(quad, 1, i + 1)
        plt.plot(cluster_data.transpose(), alpha=0.5)  # Use alpha for transparency
        #print(cluster_data)
        plt.title(f'Cluster {cluster_label} index {i}')
        plt.xlabel('Time [ms]')
        plt.ylabel('Signal Amplitude')

        # Plot the average waveform
        mean_wave = np.mean(cluster_data, axis=0)
        std_wave = np.std(cluster_data, axis=0)
        plt.plot(mean_wave, color='black', linewidth=2, label='Avg. Waveform')
        plt.legend()

    # Adjust layout to prevent overlapping
    #plt.tight_layout()
    plt.subplots_adjust(hspace=0.5)
    plt.show()
    spike_list=np.array(spike_list)
    for i in unique_labels:
        ul=spike_list[labels==i]
        final_data.append(ul)
        plt.subplot(quad, 1, i + 2)
        plt.hist(np.diff(ul), bins=100, density=True, alpha=0.5, color='blue', edgecolor='black')
        plt.title(f'ISI: Cluster {i} numerosity: {len(final_data[i+1])}, \n firing rate: {len(final_data[i+1])*10000/len_data}')
    plt.subplots_adjust(hspace=1)
    plt.show()
    return final_data


In [ ]:
cut=extra
spike_list=final_data_neg[0]
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN, HDBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.metrics import silhouette_score
from scipy.stats import kurtosis
import skfuzzy as fuzz
import numpy as np
import math

scale = StandardScaler()
estratti_norm = scale.fit_transform(cut)
print('Total spikes: ', estratti_norm.shape[0])
n_comp=10
pca = PCA(n_components=n_comp)
transformed = pca.fit_transform(estratti_norm)
print('transformed')
#transformed=cut

#dbscan = DBSCAN(eps=epsilon, min_samples=min_s, leaf_size=leafsize)
dbscan = HDBSCAN(min_cluster_size=mcs, min_samples=min_s, leaf_size=leafsize)
labels = dbscan.fit_predict(transformed)


final_data=[]
unique_labels = np.unique(labels)

if len(unique_labels) == 1:
    print("DBSCAN assigned only one cluster.")
else:
    silhouette_avg = silhouette_score(transformed, labels)
    num_clusters = len(np.unique(labels[labels != -1]))
    print("For", num_clusters,"clusters, the silhouette score is:", silhouette_avg)

cut = np.array(cut)
fig = plt.figure(figsize=(8, 10))

# Iterate over unique cluster labels
for i, cluster_label in enumerate(unique_labels):
    # Extract data points for the current cluster
    cluster_data = cut[labels == cluster_label]

    # Plot the individual cluster data
    quad = math.ceil(len(unique_labels) / 2)
    plt.subplot(quad+2, quad, i + 1)
    plt.plot(cluster_data.transpose(), alpha=0.5)  # Use alpha for transparency
    #print(cluster_data)
    plt.title(f'Cluster {cluster_label} index {i}')
    plt.xlabel('Time [ms]')
    plt.ylabel('Signal Amplitude')

    # Plot the average waveform
    mean_wave = np.mean(cluster_data, axis=0)
    std_wave = np.std(cluster_data, axis=0)
    plt.plot(mean_wave, color='black', linewidth=2, label='Avg. Waveform')
    plt.legend()

# Adjust layout to prevent overlapping
#plt.tight_layout()
plt.subplots_adjust(hspace=0.5)
plt.show()
spike_list=np.array(spike_list)
for i in unique_labels:
    ul=spike_list[labels==i]
    final_data.append(ul)
    plt.subplot(quad+2, quad, i + 2)
    plt.hist(np.diff(ul), bins=100, density=True, alpha=0.5, color='blue', edgecolor='black')
    plt.title(f'ISI: Cluster {i} numerosity: {len(final_data[i+1])}, \n firing rate: {len(final_data[i+1])*10000/len_data}')
plt.subplots_adjust(hspace=1)
plt.show()
final_data


In [ ]:
cluster_label

## Clustering

In [ ]:
final_data_pos= clus(pos_cut,'fuzzy',n_pos,3,len_data)

baseline: 2 cluster (10, 9 firing rates)
pop 2 stimulation: 2 cluster (19, 17 firing rates)

In [ ]:
final_data_neg= clus(neg_cut,'fuzzy',n_neg,2,len_data)

In [ ]:
neurons = []
#for electrode in cluster:
for neuron in final_data_pos:
    neurons.append(neuron)
for neuron in final_data_neg:
    neurons.append(neuron)
neurons

In [ ]:
data_healthy=neurons[2]
ISI_healthy = np.diff(data_healthy)/10000

## INVERSE

In [ ]:
plt.figure(figsize=(20,12))
plt.plot(data_healthy[:-1]/10000,ISI_healthy)

In [ ]:
min(ISI_healthy)

In [ ]:
bins = np.arange(0, .5, 0.001)           # Define 1 ms bins for histogram,
counts, _ = np.histogram(ISI_healthy, bins)  # ... compute histogram of the ISIs,
prob = counts / len(ISI_healthy)             # ... convert to probability,
plt.figure()                 # ... create figure and axes objects that we can reuse later,
plt.stem(bins[:-1], prob)                 # ... and plot the probabilities,                # ... with fixed x-limits,
plt.xlabel('ISI [s]')                        # ... and axes labeled.
plt.ylabel('Probability')
plt.show()

In [ ]:
with pm.Model() as model:
    mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
    lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
    obs1 = pm.Wald('like',mu=mu1,lam=lam1,observed=ISI_healthy)

    '''
    BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
    mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
    lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
    obs1 = pm.Wald('like',mu=mu1,lam=lam1,observed=ISI_healthy)
    '''
    
    step = pm.NUTS(target_accept=0.9)
    trace = pm.sample(step=step,draws=1000,chains=1,tune=1000,cores=4)

In [ ]:
map_estimate = pm.find_MAP(model=model)

map_estimate

ig_waic = pm.waic(trace, model)

ig_waic

In [ ]:
with model:
    ppc_trace = pm.sample_posterior_predictive(trace,model=model)

bins = np.arange(0, .5, 1e-3) 
plt.figure (figsize=(14,10))

hist = np.histogram(ppc_trace['posterior_predictive']['like'].values,bins=bins)
#plt.axis([0,0.3,0,160])
a= plt.hist(ISI_healthy,bins)
plt.plot(hist[1][:-1],hist[0]/1000,linewidth=3);

In [ ]:
lista_samples=[]
for i in list(ppc_trace['posterior_predictive']['like'].values):
    lista_samples.extend(i)

In [ ]:
counts, _ = np.histogram(ISI_healthy, bins) 
prob_emp = counts / np.sum(counts)
counts, _ = np.histogram(lista_samples, bins) 
prob_model = counts / np.sum(counts)

Femp = np.cumsum(prob_emp)           
Fmodel = np.cumsum(prob_model)          
plt.figure()
plt.plot(bins[:-1], Femp)                
plt.plot(bins[:-1], Fmodel, 'r')       
#plt.xlim([0, 0.2])                  
plt.xlabel('Time [s]')
plt.ylabel('CDF')
plt.legend(['Empirical','Model'])
plt.show()

In [ ]:
plt.figure()
Nlow = len(ISI_healthy)  
# Plot the confidence bounds
plt.plot([0, 1], [x + 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'r:')
plt.plot([0, 1], [x - 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'r:')
plt.plot(Femp, Fmodel)
plt.axis([0, 1, 0, 1])         
plt.xlabel('Model CDF')
plt.ylabel('Empirical CDF')
plt.show()

## MIXTURE

In [ ]:
ISI_data=ISI_healthy
with pm.Model() as model:
    ##### WALD DISTRIBUTION (INVERSE GAUSSIAN)
    mu1 = pm.Uniform('mu1',lower=0.01,upper=0.1)
    lam1 = pm.Uniform('lam1',lower=0.01,upper=0.04)
    obs1 = pm.Wald.dist(mu=mu1,lam=lam1)


    mu2 = pm.Uniform('mu2',lower=0,upper=0.2)
    sigma2 = pm.Uniform('sigma2',lower=0.0001,upper=0.5)
    obs2 = pm.TruncatedNormal.dist(mu=mu2, sigma=sigma2, lower=0.0)

    mu3 = pm.Uniform('mu3',lower=0.1,upper=0.6)
    sigma3 = pm.Uniform('sigma3',lower=0.0001,upper=0.5)
    obs3 = pm.TruncatedNormal.dist(mu=mu3, sigma=sigma3, lower=0.0)


    w = pm.Dirichlet('w', a=np.array([1., .4, .4]))

    like = pm.Mixture('like', w=w, comp_dists = [obs1, obs2, obs3], observed=ISI_data)
    map_estimate = pm.find_MAP(model=model)
    
    step = pm.NUTS(target_accept=0.9)
    trace = pm.sample(step=step,draws=1000,chains=1,tune=1000,cores=4)


In [ ]:
with model:
    ppc_trace = pm.sample_posterior_predictive(trace,model=model)

bins = np.arange(0, .5, 1e-3) 
plt.figure (figsize=(14,10))

hist = np.histogram(ppc_trace['posterior_predictive']['like'].values,bins=bins)
#plt.axis([0,0.3,0,160])
a= plt.hist(ISI_healthy,bins)
plt.plot(hist[1][:-1],hist[0]/1000,linewidth=3);

In [ ]:
lista_samples=[]
for i in list(ppc_trace['posterior_predictive']['like'].values):
    lista_samples.extend(i)

In [ ]:
counts, _ = np.histogram(ISI_healthy, bins) 
prob_emp = counts / np.sum(counts)
counts, _ = np.histogram(lista_samples, bins) 
prob_model = counts / np.sum(counts)

Femp = np.cumsum(prob_emp)           
Fmodel = np.cumsum(prob_model)          
plt.figure()
plt.plot(bins[:-1], Femp)                
plt.plot(bins[:-1], Fmodel, 'r')       
#plt.xlim([0, 0.2])                  
plt.xlabel('Time [s]')
plt.ylabel('CDF')
plt.legend(['Empirical','Model'])
plt.show()

In [ ]:
plt.figure()
Nlow = len(ISI_healthy)  
# Plot the confidence bounds
plt.plot([0, 1], [x + 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'r:')
plt.plot([0, 1], [x - 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'r:')
plt.plot(Femp, Fmodel)
plt.axis([0, 1, 0, 1])         
plt.xlabel('Model CDF')
plt.ylabel('Empirical CDF')
plt.show()

In [ ]:
len(onecut)

In [ ]:
#manual clustering based on DTW distance:
together=[]
to_remove=[]
orig=copy.deepcopy(neg_cut)
onecut=neg_cut
left=onecut
#for i in range(len(onecut)):
i=1
while i < len(onecut):
    to_remove.append(i)
    for j in range(i+1,100):#len(onecut)):
        print(i,j)
        spike1=onecut[i]
        spike2=onecut[j]
        distance,path=fastdtw(spike1,spike2)
        if distance<5.5:
            print('y')
            inds=(ind[i]-1,ind[j]-1)
            together.append(inds)
            #print(together)
            to_remove.append(j)
            #print(to_remove)
            i+=1
            k=i
            while k<le000in(onecut):
                print(k)
                pre=k-1
                inds=(ind[pre],ind[k]) 
                if inds in together:
                    i+=1
left=np.delete(left,to_remove)

together

In [ ]:
i=1
print(i)
#for i in range(len(onecut)):
nex=i+1
print(nex)
inds=(ind[i],ind[nex])
print(inds)
if inds in together:
    print(inds, 'in')
    i+=1
print(i)


In [ ]:
together

In [ ]:
type(onecut)

In [ ]:
#let's verify the dtw similarity within the identified clusters:
onecut=neg_cut
ind=n_neg
cluster=final_data_neg[0]

lt=len(cluster) * (len(cluster) - 1) // 2
pbar = tqdm(total = lt)
saved=[]
notsaved=[]
dists=[]
for k in cluster:
    for i in range(len(onecut)):
        if ind[i]==k:
            for j in range(i + 1, len(onecut)):
                if ind[j] in cluster:
                    spike1 = onecut[i]
                    spike2 = onecut[j]
                    pbar.update(1)
                    # Calculate DTW distance and alignment path
                    distance, path = fastdtw(spike1, spike2)
                    #if distance<20:
                    s=(ind[i],ind[j])
                    #saved.append(s)
                    #plt.figure()
                    #plt.plot(spike1)
                    #plt.plot(spike2)
                    #plt.show()
                    if distance>15:
                        #print(f"DTW distance between spike {i} {ind[i]},{k} and spike {j} {ind[j]}: {distance}")
                        notsaved.append(onecut[i])
                    else:
                        saved.append(onecut[i])

# Convert the 'saved' list to a numpy array if needed
notsaved = np.array(notsaved)
saved = np.array(saved)
mean=np.mean(dists)
print(mean)
saved

In [ ]:
plt.plot(notsaved.transpose())

In [ ]:
plt.plot(saved.transpose())

baseline: 2 cluster (8,9 firing rate)
pop 2 stimulation: 2 cluster (24, 26 firing rate)

In [ ]:
plt.hist(np.diff(final_data_neg[2]),bins=100, density=True, alpha=0.5, color='blue', edgecolor='black')

In [ ]:
final_data_neg[2]

from fastdtw import fastdtw
i=14
j=17
spike1=pos_cut[i]
spike2=pos_cut[j]
plt.plot(spike1)
plt.plot(spike2)
distance, path = fastdtw(spike1, spike2)

print(f"DTW distance between spike {i+1} and spike {j+1}: {distance}")
print(f"Alignment path: {path}")

In [ ]:
x=randint(1,len(pos_cut))
print(x)

plt.plot(pos_cut[x])

In [ ]:
onecut=neg_cut
ind=n_neg
cluster=final_data_neg[2]
saved=[]

l=8
k=cluster[l]

i+=1
print(ind[i],k)
if ind[i]==k:
    print('proceed')

    j=i
    
    spike1 = onecut[i]
    spike2 = onecut[j]


In [ ]:
for j in range(i + 1, len(onecut)):
    spike1 = onecut[i]
    spike2 = onecut[j]

    distance, path = fastdtw(spike1, spike2)
    if distance<6:
        s=(ind[i],ind[j])
        saved.append(s)
        print(f"DTW distance between spike {i} {ind[i]},{k} and spike {j} {ind[j]}: {distance}")
        if ind[j] in cluster:
            print(ind[j],' cointained in cluster')
        else:
            plt.figure()
            plt.plot(spike1)
            plt.plot(spike2)
            plt.show()


In [ ]:
#compute dtw for spikes 570 and 688 (not in same cluster)
#688 876
#1092 1095 1593 2304 5503 12507 125012
#1577 1801 2009 3202 4213 5211 8226
# 3070 11117 12665 16296 22891 24702 28775 30217
# 3124 51485 53426 

In [ ]:
for p,m in enumerate(final_data_neg):
    for o,n in enumerate(m):
        if n==570:
            print('ind', o,'value',n, 'cluster', m, 'number',p)

In [ ]:
final_data_neg[4]

In [ ]:
onecut=neg_cut
ind=n_neg
cluster=final_data_neg[4]

lt=len(onecut)*len(onecut)
pbar = tqdm(total = lt)
saved=[]
# Iterate through pairs of spikes and apply DTW
for k in cluster:
    for i in range(len(onecut)):
        if ind[i]==k:
            for j in range(i + 1, len(onecut)):
                spike1 = onecut[i]
                spike2 = onecut[j]
                pbar.update(1)
                # Calculate DTW distance and alignment path
                distance, path = fastdtw(spike1, spike2)
                if distance<20:
                    s=(ind[i],ind[j])
                    saved.append(s)
                    plt.figure()
                    plt.plot(spike1)
                    plt.plot(spike2)
                    plt.show()
                    print(f"DTW distance between spike {i} {ind[i]},{k} and spike {j} {ind[j]}: {distance}")
                    print(f"Alignment path: {path}")
saved

In [ ]:
final_data_pos[0]

In [ ]:
final_data_pos[1]

In [ ]:
plt.plot(pos_cut[0])

In [ ]:
final_data_pos[0]

In [ ]:
lt=len(final_data_pos[0])*len(final_data_pos[0])
pbar = tqdm(total = lt)
# Iterate through pairs of spikes and apply DTW
for i in final_data_pos[0]:
    for j in final_data_pos[0]:
        spike1 = pos_cut[i]
        spike2 = pos_cut[j]
        pbar.update(1)
        # Calculate DTW distance and alignment path
        distance, path = fastdtw(spike1, spike2)
        if distance<20:
            plt.figure()
            plt.plot(spike1)
            plt.plot(spike2)
            plt.show()
            print(f"DTW distance between spike {i+1} and spike {j+1}: {distance}")
            print(f"Alignment path: {path}")